In [135]:
import numpy as np
import pandas as pd
from datetime import datetime
import math as m

In [136]:
class UserItemData:
    def __init__(self, path, start_date=None, end_date=None, min_ratings=0):
        self.path = path
        self.start_date = start_date
        self.end_date = end_date
        self.min_ratings = min_ratings
        self.data = pd.read_csv(path, encoding="windows-1250", on_bad_lines='skip', sep='\t', skiprows = 0, parse_dates= {"date" : ["date_year","date_month","date_day"]},
                keep_date_col=True)
        
        if self.start_date is not None and self.end_date is not None:
            self.start_date = datetime.strptime(self.start_date, '%d.%m.%Y')
            self.end_date = datetime.strptime(self.end_date, '%d.%m.%Y')

            self.data = self.data[(self.data['date'] >= self.start_date) & (self.data['date'] < self.end_date)]

        self.data['Allratings'] = self.data.groupby('movieID')['movieID'].transform('size')
        if self.min_ratings > 0:
            self.data = self.data.loc[self.data['Allratings'] >= self.min_ratings]


    def nratings(self):
        return len(self.data)

uim = UserItemData('data/user_ratedmovies.dat')
print(uim.nratings())

uim = UserItemData('data/user_ratedmovies.dat', start_date = '12.1.2007', end_date='16.2.2008', min_ratings=100)
print(uim.nratings())

855598
73584


In [137]:
class MovieData:
    def __init__(self, path):
        self.path = path
        self.movies = pd.read_csv(path, encoding="windows-1250", on_bad_lines='skip', sep='\t', skiprows = 0)

    def get_title(self, movieID):
        return self.movies.loc[self.movies["id"] == movieID]["title"].values[0]

md = MovieData('data/movies.dat')
print(md.get_title(1))

Toy story


In [138]:
class RandomPredictor:
    def __init__(self, min, max):
        self.min = min
        self.max = max
        self.uim = None
    
    def fit(self, X):
        self.uim = X

    def predict(self, userID, rec_seen=True):
        if self.uim is not None:
            if not rec_seen:
                df = self.uim.data.loc[self.uim.data["userID"] == userID]['movieID']
                movieID = pd.Series(list(df))
                movieID.name = "moviesID"
                #self.uim.data = self.uim.data[~self.uim.data.index.isin(movieID)]
                self.uim.data = self.uim.data[~self.uim.data['movieID'].isin(movieID)]
                    
            movies = np.array(self.uim.data["movieID"])
            numbers = np.random.randint(self.min, self.max+1, len(movies))
            res = {}
            for i in range(len(movies)):
                res[movies[i]] = numbers[i] 

            return res
        else:
            print("none")

md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rp.fit(uim)
pred = rp.predict(78)
print(type(pred))
items = [1, 3, 20, 50, 100]
for item in items:
    print("Film: {}, ocena: {}".format(md.get_title(item), pred[item]))

<class 'dict'>
Film: Toy story, ocena: 4
Film: Grumpy Old Men, ocena: 1
Film: Money Train, ocena: 2
Film: The Usual Suspects, ocena: 2
Film: City Hall, ocena: 5


In [139]:
class Recommender:
    def __init__(self, predictor):
        self.predictor = predictor
        self.uim = None

    def fit(self, X):
        self.uim = X
        self.predictor.fit(X)

    def recommend(self, userID, n = 10, rec_seen = True):
        return dict(sorted(self.predictor.predict(userID, rec_seen).items(), key=lambda item: item[1], reverse=True)[:n])


md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items.items():
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))  

Film: Beverly Hills Cop III, ocena: 5
Film: Die Hard, ocena: 5
Film: Armageddon, ocena: 5
Film: South Park: Bigger Longer & Uncut, ocena: 5
Film: Unbreakable, ocena: 5


In [140]:
class AveragePredictor:
    def __init__(self, b):
        if b >= 0:
            self.b = b
        else:
            self.b = 0

    def fit(self, X):
        self.uim = X
        self.uim.data['n'] = self.uim.data.groupby('movieID')['movieID'].transform('size')
        self.uim.data['vs'] = self.uim.data.groupby('movieID')['rating'].transform('sum')
        self.g_avg = self.uim.data['rating'].mean()
        # avg = (vs + b * g_avg) / (n + b
        self.uim.data['avg'] = (self.uim.data['vs'] + (self.b * self.g_avg)) / (self.uim.data['n'] + self.b)

    def predict(self, userID, rec_seen=True):
        if not rec_seen:
            df = self.uim.data.loc[self.uim.data["userID"] == userID]['movieID']
            movieID = pd.Series(list(df))
            movieID.name = 'movieIDs'
            self.uim.data = self.uim.data[~self.uim.data['movieID'].isin(movieID)]
            
        movies = list(self.uim.data["movieID"])
        avg = list(self.uim.data["avg"])

        res = {}
        for i in range(len(movies)):
            res[movies[i]] = avg[i] 

        return res


md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = AveragePredictor(100)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items.items():
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))  

Film: The Usual Suspects, ocena: 4.225944245560473
Film: The Godfather: Part II, ocena: 4.146907937910189
Film: Cidade de Deus, ocena: 4.116538340205236
Film: The Dark Knight, ocena: 4.10413904093503
Film: 12 Angry Men, ocena: 4.103639627096175


In [141]:
class ViewsPredictor:
    def __init__(self):
        pass

    def fit(self, X):
        self.uim = X

    def predict(self, userID, rec_seen=True):
        if not rec_seen:
            df = self.uim.data.loc[self.uim.data["userID"] == userID]["movieID"]
            movies = pd.Series(list(df))
            movies.name = "moviesID"
            #self.uim.data = self.uim.data[~self.uim.data.index.isin(movies)]
            self.uim.data = self.uim.data.loc[~self.uim.data['movieID'].isin(movies)]

        return dict(self.uim.data.groupby("movieID").count()['userID'])

md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = ViewsPredictor()
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items.items():
   print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))  

Film: The Lord of the Rings: The Fellowship of the Ring, ocena: 1576
Film: The Lord of the Rings: The Two Towers, ocena: 1528
Film: The Lord of the Rings: The Return of the King, ocena: 1457
Film: The Silence of the Lambs, ocena: 1431
Film: Shrek, ocena: 1404


In [142]:
import warnings

warnings.filterwarnings('ignore')
class ItemBasedPredictor:
    def __init__(self, min_values = 0, threshold = 0):
        self.min_values = min_values
        self.threshold = threshold

    def fit(self, X):
        self.uim = X

    def similarity(self, p1, p2):
        movieIDs = list(self.uim.data['movieID'].unique())
        self.matrika = pd.DataFrame(0, index=movieIDs, columns=movieIDs)

        self.avg = self.uim.data.groupby('userID')['rating'].mean()

        u1 = list(self.matrika.index)
        u2 = list(self.matrika.columns)
    
        df1 = self.uim.data.loc[(self.uim.data["movieID"] == p1)]
        df2 = self.uim.data.loc[(self.uim.data["movieID"] == p2)]

        df = pd.merge(df1, df2, how="inner", on=["userID"])
        df = pd.merge(df, self.avg, how="inner", on=["userID"])

        # Če imamo minimalno toliko uporabnikov
        if len(df) >= self.min_values:
            Ru = np.array(df['rating'])
            R1 = np.array(df['rating_x'])
            R2 = np.array(df['rating_y'])

            sub1 = np.subtract(R1, Ru)
            sub2 = np.subtract(R2, Ru)
            produkt1 = np.multiply(sub1, sub2)
            vsota1 = np.sum(produkt1)

            sqrt_vsota_square1 = np.sqrt(np.sum(np.square(sub1)))
            sqrt_vsota_square2 = np.sqrt(np.sum(np.square(sub2)))
            produkt_final = sqrt_vsota_square1 * sqrt_vsota_square2

            result = vsota1 / produkt_final

            if result < 0 or result < self.threshold:
                result = 0
        else:
            result = 0
        self.matrika.loc[p1, p2] = result

        return self.matrika.loc[p1, p2]

    def predict(self, userID, rec_seen = True):
        # Vsi filmi
        moviesID = self.uim.data.drop_duplicates('movieID')['movieID']
        # Filmi userja z userID
        if not rec_seen:
            movies_user = self.uim.data.loc[self.uim.data["userID"] == userID]
            
        result = {}
        for id in moviesID:
            #similarities for each movie
            zgoraj = 0
            spodaj = 0
            for ind, row in movies_user.iterrows():
                u_rating = row['rating']
                movie2 = row['movieID']

                zgoraj += self.similarity(id, movie2) * u_rating
                spodaj += self.similarity(id, movie2)

            # Na mestu filma shranimo količnik
            result[id] = zgoraj / spodaj
        return result

    
    def vrni_matriko(self):
        movieIDs = list(self.uim.data['movieID'].unique())
        self.matrika = pd.DataFrame(0, index=movieIDs, columns=movieIDs)

        self.avg = self.uim.data.groupby('userID')['rating'].mean()

        u1 = list(self.matrika.index)
        u2 = list(self.matrika.columns)
    
        for i in u1:
            for j in u2:

                df1 = self.uim.data.loc[(self.uim.data["movieID"] == i)]
                df2 = self.uim.data.loc[(self.uim.data["movieID"] == j)]

                df = pd.merge(df1, df2, how="inner", on=["userID"])
                df = pd.merge(df, self.avg, how="inner", on=["userID"])

                # Če imamo minimalno toliko uporabnikov
                if len(df) >= self.min_values:
                    Ru = np.array(df['rating'])
                    R1 = np.array(df['rating_x'])
                    R2 = np.array(df['rating_y'])

                    sub1 = np.subtract(R1, Ru)
                    sub2 = np.subtract(R2, Ru)
                    produkt1 = np.multiply(sub1, sub2)
                    vsota1 = np.sum(produkt1)

                    sqrt_vsota_square1 = np.sqrt(np.sum(np.square(sub1)))
                    sqrt_vsota_square2 = np.sqrt(np.sum(np.square(sub2)))
                    produkt_final = sqrt_vsota_square1 * sqrt_vsota_square2

                    result = vsota1 / produkt_final

                    if result < 0 or result < self.threshold:
                        result = 0
                else:
                    result = 0
                    
                self.matrika.loc[i, j] = result

        return self.matrika

md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)
print("Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716): ", rp.similarity(1580, 2716))
print("Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527): ", rp.similarity(1580, 527))
print("Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780): ", rp.similarity(1580, 780))

Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716):  0.23395523176756633
Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527):  0
Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780):  0.42466125844687613


In [143]:
print("Predictions for 78: ")
rec_items = rec.recommend(78, n=15, rec_seen=False)
for idmovie, val in rec_items.items():
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

Predictions for 78: 
Film: Twelve Monkeys, ocena: 4.4678775914494135
Film: Lost in Translation, ocena: 4.44968981828876
Film: Alien, ocena: 4.44792924555423
Film: Groundhog Day, ocena: 4.434545789386526
Film: Memento, ocena: 4.434159144102171
Film: Eternal Sunshine of the Spotless Mind, ocena: 4.426170462610086
Film: Being John Malkovich, ocena: 4.424690175230734
Film: Fargo, ocena: 4.403136853866336
Film: American Beauty, ocena: 4.40090030407282
Film: Le fabuleux destin d'Amélie Poulain, ocena: 4.397086146284772
Film: Blade Runner, ocena: 4.391002451402141
Film: Monty Python and the Holy Grail, ocena: 4.3799898927755905
Film: Shichinin no samurai, ocena: 4.355734790310159
Film: The Usual Suspects, ocena: 4.354681728067836
Film: Reservoir Dogs, ocena: 4.352819064017404


In [144]:
df = rp.vrni_matriko()

l = []
slovar = {}
for i in sorted(df.values.flatten(), reverse=True):
    if i < 0.99:
        l.append(i)    

slovar = {}
for k in l[:20]:
    for i in df.index:
        for j in df.columns:
            if k == df[i][j]:
                terka = (i, j)
                slovar[terka] = k


for key, val in slovar.items():
    md = MovieData('data/movies.dat')
    print("Film1: {}, Film2: {}, podobnost: {}".format(md.get_title(key[0]), md.get_title(key[1]), val))

Film1: The Lord of the Rings: The Two Towers, Film2: The Lord of the Rings: The Return of the King, podobnost: 0.8439842148481417
Film1: The Lord of the Rings: The Return of the King, Film2: The Lord of the Rings: The Two Towers, podobnost: 0.8439842148481417
Film1: The Lord of the Rings: The Fellowship of the Ring, Film2: The Lord of the Rings: The Two Towers, podobnost: 0.8231885401761888
Film1: The Lord of the Rings: The Two Towers, Film2: The Lord of the Rings: The Fellowship of the Ring, podobnost: 0.8231885401761888
Film1: The Lord of the Rings: The Fellowship of the Ring, Film2: The Lord of the Rings: The Return of the King, podobnost: 0.8079374897442496
Film1: The Lord of the Rings: The Return of the King, Film2: The Lord of the Rings: The Fellowship of the Ring, podobnost: 0.8079374897442496
Film1: Kill Bill: Vol. 2, Film2: Kill Bill: Vol. 2, podobnost: 0.7372340224381029
Film1: Kill Bill: Vol. 2, Film2: Kill Bill: Vol. 2, podobnost: 0.7372340224381029
Film1: Star Wars, Film2: